# Spark DataFrame - Basics

Let's start off with the fundamentals of Spark DataFrame. 

Objective: In this exercise, you'll find out how to start a spark session, read in data, explore the data and manipuluate the data (using DataFrame syntax as well as SQL syntax). Let's get started! 

In [1]:
# Must be included at the beginning of each new notebook. Remember to change the app name.
import findspark
findspark.init('/home/ubuntu/spark-3.2.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('basics').getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/31 00:21:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# Let's read in the data. Note that it's in the format of JSON.
df = spark.read.json('Datasets/people.json')

## Data Exploration

In [3]:
# The show method allows you visualise DataFrames. We can see that there are two columns. 
df.show()

# You could also try this. 
df.columns

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



['age', 'name']

In [4]:
# We can use the describe method get some general statistics on our data too. Remember to show the DataFrame!
# But what about data type?
df.describe().show()

+-------+------------------+-------+
|summary|               age|   name|
+-------+------------------+-------+
|  count|                 2|      3|
|   mean|              24.5|   null|
| stddev|7.7781745930520225|   null|
|    min|                19|   Andy|
|    max|                30|Michael|
+-------+------------------+-------+



In [5]:
# For type, we can use print schema. 
# But wait! What if you want to change the format of the data? Maybe change age to an integer instead of long?
df.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



## Data Manipulation

In [6]:
# Let's import in the relevant types.
from pyspark.sql.types import (StructField,StringType,IntegerType,StructType)

In [7]:
# Then create a variable with the correct structure.
data_schema = [StructField('age',IntegerType(),True),
              StructField('name',StringType(),True)]

final_struct = StructType(fields=data_schema)

In [8]:
# And now we can read in the data using that schema. If we print the schema, we can see that age is now an integer. 
df = spark.read.json('Datasets/people.json', schema=final_struct)

df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- name: string (nullable = true)



In [9]:
# We can also select various columns from a DataFrame. 
df.select('age').show()

# We could split up these steps, first assigning the output to a variable, then showing that variable. As you see, the output is the same.
ageColumn = df.select('age')

ageColumn.show()

+----+
| age|
+----+
|null|
|  30|
|  19|
+----+

+----+
| age|
+----+
|null|
|  30|
|  19|
+----+



In [10]:
# We can also add columns, manipulating the DataFrame.

df.withColumn('double_age',df['age']*2).show()

# But note that this doesn't alter the original DataFrame. You need to assign the output to a new variable in order to do so.
df.show()

+----+-------+----------+
| age|   name|double_age|
+----+-------+----------+
|null|Michael|      null|
|  30|   Andy|        60|
|  19| Justin|        38|
+----+-------+----------+

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [11]:
# We can rename columns too! 
df.withColumnRenamed('age', 'my_new_age').show()

+----------+-------+
|my_new_age|   name|
+----------+-------+
|      null|Michael|
|        30|   Andy|
|        19| Justin|
+----------+-------+



## Introducing SQL
We can query a DataFrame as if it were a table! Let's see a few examples of that below:

In [12]:
# First, we have to register the DataFrame as a SQL temporary view.
df.createOrReplaceTempView('people')

# After that, we can use the SQL programming language for queries. 
results = spark.sql("SELECT * FROM people")

In [13]:
# Here's another example:
results = spark.sql("SELECT age FROM people WHERE age >= 19")
results.show()

+---+
|age|
+---+
| 30|
| 19|
+---+



Now that we're done with this tutorial, let's move on to Spark DataFrame Operations!